This code loops through all the teams in college basketball form years 2010,2011,2012 and stores the results in CSV.  
For some reason 2009 is not working using `sportsipy` packages. May need to just extract it manually.

**17-08-22:**. 
The API does not work well when player changes teams. It just seems to pick up the last team and repeat that value.  
The stats are correct and conference is correct, just the school name is not correct.  
e.g. `player_id == 'anali-okoloji-1'`

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from lxml import etree, html
import os
import numpy as np
from datetime import datetime
from sportsipy.ncaab.teams import Teams
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_colwidth", 200)

**NCAA Loop - get all player ids**  
Done for 2010,2011,2012   
Done for 2013,2014,2015,2016

In [ ]:
years = [2013,2014,2015,2016]
for year in years:
    teams = Teams(year = year)
    team_lst = [team for team in teams] # all teams in a list allows me to select just a subset of teams for testing

    df = pd.DataFrame()

    for i, team in enumerate(team_lst):
        print(str(year)+':'+str(i)+' of '+str(len(team_lst))+':'+ team.name +'start at: '+datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
        roster = team.roster  # Gets each team's roster
        for player in roster.players:
            player_df = pd.concat([player.dataframe.reset_index(),pd.DataFrame({'name':[player.name]*player.dataframe.shape[0]})],axis = 1) # add name to df
            df = pd.concat([df,player_df],axis = 0)
    # create csv
    df.to_csv('cbb_'+str(year)+'.csv',index = False)

## Create tables from output
* when you get new data, just run the whole thing again

**Create table player_cbb**

Separate into:
1. cbb_player: player_id, name
2. cbb_player_stats: player_id, school, stats...
3. cbb_player_career_stats: player_id, career stats
4. cbb_player_school: player_id, school details, season

**Create cbb_player**

In [146]:
files = os.listdir('01_raw_csv_files')
files_csv = [f for f in files if f[-3:] == 'csv' and f[:3]=='cbb']

cbb_player = pd.DataFrame()
for file in files_csv:
    df = pd.read_csv('01_raw_csv_files/'+file)
    cbb_player = pd.concat([cbb_player,df.loc[:,['player_id','name']].drop_duplicates().reset_index(drop=True)])
cbb_player = cbb_player.drop_duplicates().reset_index(drop= True)    

In [147]:
cbb_player.to_csv('02_database/cbb_player.csv',index=False)

**Create cbb_player_stats**

In [148]:
files = os.listdir('01_raw_csv_files')
files_csv = [f for f in files if f[-3:] == 'csv' and f[:3]=='cbb']

cbb_player_stats = pd.DataFrame()
for file in files_csv:
    df = pd.read_csv('01_raw_csv_files/'+file)
    cbb_player_stats = pd.concat([cbb_player_stats,df.loc[(df.level_0 != 'Career'),:].drop(columns = 'name')])
cbb_player_stats = cbb_player_stats.drop_duplicates().reset_index(drop= True)   
cbb_player_stats.rename(columns = {'level_0':'season'},inplace = True)

In [149]:
cbb_player_stats.to_csv('02_database/cbb_player_stats.csv',index=False)

**Create cbb_player_career_stats**  
At the moment this seems to be a table that will need to be updated since there is no season variable.  
So for a Freshman, this table will need to be updated the following year. Not sure if this will be a useful table

In [150]:
files = os.listdir('01_raw_csv_files')
files_csv = [f for f in files if f[-3:] == 'csv' and f[:3]=='cbb']

cbb_player_career_stats = pd.DataFrame()
for file in files_csv:
    df = pd.read_csv('01_raw_csv_files/'+file)
    cbb_player_career_stats = pd.concat([cbb_player_career_stats,df.loc[(df.level_0 == 'Career'),:].drop(columns = 'name')])
cbb_player_career_stats = cbb_player_career_stats.drop_duplicates().reset_index(drop= True)   
cbb_player_career_stats.rename(columns = {'level_0':'season'},inplace = True)

In [151]:
cbb_player_career_stats.to_csv('02_database/cbb_player_career_stats.csv',index=False)

**Create table cbb_player_school**

In [152]:
files = os.listdir('01_raw_csv_files')
files_csv = [f for f in files if f[-3:] == 'csv' and f[:3]=='cbb']

cbb_player_school = pd.DataFrame()
for file in files_csv:
    df = pd.read_csv('01_raw_csv_files/'+file)
    cbb_player_school = pd.concat([cbb_player_school,df.loc[(df.level_0 != 'Career'),['player_id','level_0','conference','team_abbreviation']]])
cbb_player_school = cbb_player_school.drop_duplicates().reset_index(drop= True)   
cbb_player_school.rename(columns = {'level_0':'season'},inplace = True)

In [153]:
cbb_player_school.to_csv('02_database/cbb_player_school.csv',index=False)